In [1]:
# Modules: gdal/3.6.4  
# Environment base: /g/data/xe2/John/geospatenv

In [2]:
# Note: For the canopy_height.download_new_tiles to work, you need to create an AWS account, then create a file named .aws/credentials in your home directory (e.g. /home/147/cb8590) with these contents:
# [default]
# aws_access_key_id = ACCESS_KEY
# aws_secret_access_key = SECRET_KEY

In [3]:
# Standard library
import os

# Local imports
os.chdir(os.path.join(os.path.expanduser('~'), "Projects/PaddockTS"))
from DAESIM_preprocess.util import gdata_dir, scratch_dir
from DAESIM_preprocess.terrain_tiles import terrain_tiles
from DAESIM_preprocess.slga_soils import slga_soils, asris_urls
from DAESIM_preprocess.ozwald_yearly import ozwald_yearly_average
from DAESIM_preprocess.ozwald_8day import ozwald_8day, ozwald_8day_abbreviations
from DAESIM_preprocess.ozwald_daily import ozwald_daily, ozwald_daily_abbreviations
from DAESIM_preprocess.silo_daily import silo_daily
from DAESIM_preprocess.canopy_height import canopy_height

In [4]:
# Choosing location
lat, lon = -34.3890427, 148.469499
buffer = 0.005  # 0.01 degrees is about 1km in each direction, so 2km total
stub = "Test"
start_year = 2020
end_year = 2021

# Specify output destinations
outdir = os.path.join(gdata_dir, "Data/PadSeg/")
tmp_dir = scratch_dir

In [5]:
%%time
# Download elevation from terrain tiles
terrain_tiles(lat, lon, buffer, outdir, stub, tmp_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'gdalwarp'

In [6]:
%%time
# The CSIRO soils API seems to frequently be randomly unavailable unfortunately
variables = ['Clay', 'Sand', 'Silt', 'pH_CaCl2']
slga_soils(variables, lat, lon, buffer, outdir, stub)

Downloaded /g/data/xe2/cb8590/Data/PadSeg/Test_Clay.tif
Downloaded /g/data/xe2/cb8590/Data/PadSeg/Test_Sand.tif
Failed to download Silt
Downloaded /g/data/xe2/cb8590/Data/PadSeg/Test_Silt.tif
Downloaded /g/data/xe2/cb8590/Data/PadSeg/Test_pH_CaCl2.tif
CPU times: user 209 ms, sys: 38.1 ms, total: 247 ms
Wall time: 1min 5s


In [7]:
%%time
# Download average climate data from SILO hosted on OzWald
variables = ["Tmax", "Tmin", "Pg"]
ozwald_yearly_average(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Saved: /g/data/xe2/cb8590/Data/PadSeg/Test_Maximum_Temperature_2020_2021_average.tif
Saved: /g/data/xe2/cb8590/Data/PadSeg/Test_Minimum_Temperature_2020_2021_average.tif
Saved: /g/data/xe2/cb8590/Data/PadSeg/Test_Annual_Rainfall_2020_2021_average.tif
CPU times: user 284 ms, sys: 162 ms, total: 446 ms
Wall time: 2.57 s


In [8]:
%%time
# Download modelled variables from OzWald
variables = ['Ssoil']
ds = ozwald_8day(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Downloaded /scratch/xe2/cb8590/Test_Ssoil_2020.nc
Downloaded /scratch/xe2/cb8590/Test_Ssoil_2021.nc
Saved: /g/data/xe2/cb8590/Data/PadSeg/Test_ozwald_8day.nc
CPU times: user 439 ms, sys: 164 ms, total: 603 ms
Wall time: 4.6 s


In [9]:
%%time
# Download daily weather data hosted on OzWald
variables = ["VPeff", "Uavg"]
ds = ozwald_daily(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Downloaded /scratch/xe2/cb8590/Test_VPeff_2020.nc
Downloaded /scratch/xe2/cb8590/Test_VPeff_2021.nc
Downloaded /scratch/xe2/cb8590/Test_Uavg_2020.nc
Downloaded /scratch/xe2/cb8590/Test_Uavg_2021.nc
Saved: /g/data/xe2/cb8590/Data/PadSeg/Test_ozwald_daily.nc
CPU times: user 115 ms, sys: 4.16 ms, total: 120 ms
Wall time: 1.59 s


In [10]:
%%time
# Load data pre-downloaded to gdata from SILO's AWS server
ds_silo_daily = silo_daily(["daily_rain", "max_temp", "min_temp", "et_morton_actual", "et_morton_potential"], lat, lon, buffer, start_year, end_year, outdir, stub)
ds_silo_daily

Saved: /g/data/xe2/cb8590/Data/PadSeg/Test_silo_daily.nc
CPU times: user 230 ms, sys: 3.22 s, total: 3.45 s
Wall time: 8.54 s


<xarray.Dataset>
Dimensions:              (time: 731)
Coordinates:
    lat                  float64 -34.4
    lon                  float64 148.4
  * time                 (time) datetime64[ns] 2020-01-01 ... 2021-12-31
Data variables:
    daily_rain           (time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    crs                  (time) |S1 b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    max_temp             (time) float32 34.1 36.1 39.8 44.0 ... 26.5 29.8 31.3
    min_temp             (time) float32 16.3 14.5 17.8 19.6 ... 8.2 9.5 10.6
    et_morton_actual     (time) float32 0.9 4.0 4.3 0.8 0.1 ... 4.2 5.3 5.7 6.0
    et_morton_potential  (time) float32 11.9 10.6 12.1 14.0 ... 7.6 8.4 8.2 8.7
Attributes:
    department:               Department of Environment and Science
    department_short:         DES
    copyright:                Copyright - the State of Queensland Department ...
    site_url:                 http://www.longpaddock.qld.gov.au
    institution:              Queensland Government, Department of Environmen...
    raster_source:            Gridded surface was created by interpolating ob...
    raster_source_additions:  and other suppliers (see the SILO webpage for d...
    metadata_url:             http://qldspatial.information.qld.gov.au/catalo...
    reference:                Jeffrey, S.J., Carter, J.O., Moodie, K.B.A. and...
    disclaimer:               1. The user accepts all responsibility and risk...

In [11]:
%%time 
# Download and merge/crop canopy height tiles from their AWS server
canopy_height(lat, lon, buffer, outdir, stub, tmp_dir)

Saved: /g/data/xe2/cb8590/Data/PadSeg/Test_canopy_height.tif
CPU times: user 23.6 s, sys: 511 ms, total: 24.1 s
Wall time: 26.3 s
